In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


### Compiler Tests for Exception

some WebAssembly code examples available(try, catch, throw):   
https://github.com/WebAssembly/wabt/tree/main/test/parse/expr

### Explicit Exception:
- parse "throw i" where i is integer exception tag
- parse "try catch"

In [ ]:
# "Throw" can be parsed
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 
    then throw 39 
    else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program test
    var a: integer
        a ← sqrt(15)
        write(a)
""")

In [13]:
#"try-catch" can be parsed
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 
    then throw 3
    else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program test
    var x, a: integer
    try
        x ← read()
        a ← sqrt(x)
        write(a)
    catch 3
        write(-1)
    catchall
""", 'trycatch.wat')

In [14]:
!wat2wasm --enable-exceptions trycatch.wat

In [16]:
#runwasm('trycatch.wasm')

<IPython.core.display.Javascript object>

In [17]:
# multiple catch can be parsed
compileString("""
procedure foo(x: integer) → (r: integer)
    if x < 0 then throw 0
    else 
        if x < 10 then throw 10
        else r := x 
program test
    var x, a: integer
    try
        x ← read()
        a ← foo(x)
        write(a)
    catch 0
        write(-1)
    catch 10
        write(5)
    catchall
""", 'multicatch.wat')

In [18]:
!wat2wasm --enable-exceptions multicatch.wat

In [21]:
#runwasm('multicatch.wasm')

<IPython.core.display.Javascript object>

In [ ]:
#"try-catch" can be applied
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 
    then throw 39 
    else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch 4
                done := true
            catchall
""")

### Implicit Exception

#### Index Out of Bound:
exception tag to be thrown is 110

In [9]:
# Implicit Exception: Index out of bound (exception tag 110)
# == if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""
var a: [1..2] → integer
program fool
    var i: integer
    a[1] := 1; a[2] := 2
    try
        i ← read()
        write(a[i])
    catch 110
        write(110)
    catchall
""", 'badindex.wat')

In [10]:
!wat2wasm --enable-exceptions badindex.wat

In [12]:
#runwasm("badindex.wasm")

<IPython.core.display.Javascript object>

#### Div or Mod by 0:
- Div by 0: exception tag to be thrown is 111
- Mod by 0: exception tag to be thrown is 112

In [22]:
# Implicit Exception: Division by 0 (exception tag 111)
compileString("""
program fool
    var a, x: integer
    try
        x ← read()
        a := 10 div x
        write(a)
    catch 111
        write(111)
    catchall
""", 'divby0.wat')

In [23]:
!wat2wasm --enable-exceptions divby0.wat

In [24]:
#runwasm("divby0.wasm")

<IPython.core.display.Javascript object>

In [25]:
# Implicit Exception: Mod by 0 (exception tag 112)
compileString("""
program fool
    var a, x: integer
    try
        x ← read()
        a := 10 mod x
        write(a)
    catch 112
        write(112)
    catchall
""", 'modby0.wat')

In [26]:
!wat2wasm --enable-exceptions modby0.wat

In [27]:
#runwasm("modby0.wasm")

<IPython.core.display.Javascript object>